# Cool-Chic implemented manually for a single image

## Goals
The goal of this notebook is to get Cool-Chic implementation in a manual way such that all the components are clear and visible as a first step towards modifying them to lossless version.



In [3]:
import glob
import os
import subprocess
import math
from typing import Any, Dict, List
import numpy as np
from PIL import Image

DATASET_PATH = f"{os.getcwd()}/../datasets/kodak"
IMAGE_PATHS = sorted(glob.glob(f"{DATASET_PATH}/*.png"), key=lambda x: int(os.path.basename(x).split('.')[0][len("kodim"):]))
IMAGES = [np.array(Image.open(path)) for path in IMAGE_PATHS]
print(IMAGE_PATHS)

import torch
import torch.nn as nn
from pathlib import Path
import sys

from enc.component.coolchic import CoolChicEncoder
from enc.component.frame import FrameEncoder
from enc.training.train import train
from enc.training.test import test
from enc.training.loss import loss_function
from enc.utils.misc import mem_info
from enc.io.io import load_frame_data_from_file
from enc.component.core import synthesis, arm, upsampling

# Load the first Kodak image
image_path = IMAGE_PATHS[0]  # kodim01.png
print(f"Working with: {image_path}")

# Convert to Cool-Chic format
frame_data = load_frame_data_from_file(image_path, 0)
print(f"Image shape: {frame_data.data.shape}")
print(f"Image data type: {frame_data.frame_data_type}")


['/home/jakub/ETH/2025_2026_fall/thesis/Cool-Chic/coolchic/../datasets/kodak/kodim01.png', '/home/jakub/ETH/2025_2026_fall/thesis/Cool-Chic/coolchic/../datasets/kodak/kodim02.png', '/home/jakub/ETH/2025_2026_fall/thesis/Cool-Chic/coolchic/../datasets/kodak/kodim03.png', '/home/jakub/ETH/2025_2026_fall/thesis/Cool-Chic/coolchic/../datasets/kodak/kodim04.png', '/home/jakub/ETH/2025_2026_fall/thesis/Cool-Chic/coolchic/../datasets/kodak/kodim05.png', '/home/jakub/ETH/2025_2026_fall/thesis/Cool-Chic/coolchic/../datasets/kodak/kodim06.png', '/home/jakub/ETH/2025_2026_fall/thesis/Cool-Chic/coolchic/../datasets/kodak/kodim07.png', '/home/jakub/ETH/2025_2026_fall/thesis/Cool-Chic/coolchic/../datasets/kodak/kodim08.png', '/home/jakub/ETH/2025_2026_fall/thesis/Cool-Chic/coolchic/../datasets/kodak/kodim09.png', '/home/jakub/ETH/2025_2026_fall/thesis/Cool-Chic/coolchic/../datasets/kodak/kodim10.png', '/home/jakub/ETH/2025_2026_fall/thesis/Cool-Chic/coolchic/../datasets/kodak/kodim11.png', '/home/ja

In [7]:
TEST_WORKDIR = f'{os.getcwd()}/test-workdir/'
PATH_COOL_CHIC_CFG = f'{os.getcwd()}/../cfg/'

cmd = (
    f"python3 encode.py "
    f'--input={IMAGE_PATHS[0]} '
    f'--output={TEST_WORKDIR} '
    f'--workdir={TEST_WORKDIR} '
    f'--lmbda=0.001 '
    f'--job_duration_min=-1 '
    f'--print_detailed_archi '
    f'--print_detailed_struct '
    f'--enc_cfg={PATH_COOL_CHIC_CFG}enc/intra/fast_10k.cfg '
    
    f'--n_frames={1} '
)
subprocess.call(cmd, shell=True)

Namespace(input='/home/jakub/ETH/2025_2026_fall/thesis/Cool-Chic/coolchic/../datasets/kodak/kodim01.png', output='/home/jakub/ETH/2025_2026_fall/thesis/Cool-Chic/coolchic/test-workdir/', workdir='/home/jakub/ETH/2025_2026_fall/thesis/Cool-Chic/coolchic/test-workdir/', lmbda=0.001, job_duration_min=-1, print_detailed_archi=True, print_detailed_struct=True, intra_pos='0', p_pos='', n_frames=1, frame_offset=0, coding_idx=0, enc_cfg='/home/jakub/ETH/2025_2026_fall/thesis/Cool-Chic/coolchic/../cfg/enc/intra/fast_10k.cfg', dec_cfg_residue=None, dec_cfg_motion=None, start_lr=0.01, n_itr=10000, n_itr_pretrain_motion=1000, n_train_loops=1, preset='c3x_intra', layers_synthesis_residue='48-1-linear-relu,X-1-linear-none,X-3-residual-relu,X-3-residual-none', layers_synthesis_motion='9-1-linear-relu,X-1-linear-none,X-3-residual-none', arm_residue='16,2', arm_motion='8,1', n_ft_per_res_residue='1,1,1,1,1,1,1', n_ft_per_res_motion='0,0,1,1,1,1,1', ups_k_size_residue=8, ups_k_size_motion=8, ups_preconc

W0806 22:35:25.411000 162354 torch/_inductor/utils.py:1250] [0/0] Not enough SMs to use max_autotune_gemm mode


loss       nn_bpp     psnr_db    latent_bpp rate_bpp   time_sec   itr        lr         optim      patience   q_type     sr_temp    n_type     noise      record     
1.673070   0.000000   32.113313  1.058362   1.058362   160.9      2900       0.0100     all        49         softround  0.300      gaussian     0.25                  
1.631976   0.000000   32.290493  1.041842   1.041842   164.1      3000       0.0100     all        48         softround  0.298      gaussian     0.25                  
1.617723   0.000000   32.307532  1.029900   1.029900   166.8      3100       0.0100     all        47         softround  0.296      gaussian     0.25                  
1.601879   0.000000   32.377862  1.023498   1.023498   169.4      3200       0.0100     all        50         softround  0.294      gaussian     0.25       +0.080 bpp +0.565 db 
1.594357   0.000000   32.387066  1.017201   1.017201   172.0      3300       0.0100     all        50         softround  0.292      gaussian     0.24   

rm: cannot remove '/home/jakub/ETH/2025_2026_fall/thesis/Cool-Chic/coolchic/test-workdir/': Is a directory
rm: cannot remove '/home/jakub/ETH/2025_2026_fall/thesis/Cool-Chic/coolchic/test-workdir/': Is a directory
/bin/sh: 1: cannot create /home/jakub/ETH/2025_2026_fall/thesis/Cool-Chic/coolchic/test-workdir/: Is a directory
/bin/sh: 1: cannot create /home/jakub/ETH/2025_2026_fall/thesis/Cool-Chic/coolchic/test-workdir/: Is a directory
/bin/sh: 1: cannot create /home/jakub/ETH/2025_2026_fall/thesis/Cool-Chic/coolchic/test-workdir/: Is a directory
/bin/sh: 1: cannot create /home/jakub/ETH/2025_2026_fall/thesis/Cool-Chic/coolchic/test-workdir/: Is a directory
/bin/sh: 1: cannot create /home/jakub/ETH/2025_2026_fall/thesis/Cool-Chic/coolchic/test-workdir/: Is a directory
/bin/sh: 1: cannot create /home/jakub/ETH/2025_2026_fall/thesis/Cool-Chic/coolchic/test-workdir/: Is a directory
/bin/sh: 1: cannot create /home/jakub/ETH/2025_2026_fall/thesis/Cool-Chic/coolchic/test-workdir/: Is a direc

0

In [ ]:
import argparse

def get_coding_structure_from_args(args: argparse.Namespace) -> Dict[str, Any]:
    """Perform some check on the argparse object used to collect the command
    line parameters. Return a dictionary ready to be plugged into the
    ``CodingStructure`` constructor.

    Args:
        args (argparse.Namespace): Command-line argument parser.

    Returns:
        Dict[str, Any]: Dictionary ready to be plugged into the ``CodingStructure``
            constructor.
    """

    coding_structure_config = {
        "seq_name": os.path.basename(args.input).split(".")[0] if "input" in args else "",
    }
    return coding_structure_config

In [ ]:
# Software Name: Cool-Chic
# SPDX-FileCopyrightText: Copyright (c) 2023-2025 Orange
# SPDX-License-Identifier: BSD 3-Clause "New"
#
# This software is distributed under the BSD-3-Clause license.
#
# Authors: see CONTRIBUTORS.md


import os
import sys

import configargparse
import torch
from enc.component.coolchic import CoolChicEncoderParameter
from enc.component.frame import load_frame_encoder
from enc.component.intercoding.warp import WarpParameter
from enc.component.video import (
    FrameEncoderManager,
    _get_frame_path_prefix,
    encode_one_frame,
)
from enc.utils.codingstructure import CodingStructure
from enc.utils.parsecli import (
    get_coding_structure_from_args,
    get_coolchic_param_from_args,
    get_manager_from_args,
    get_warp_param_from_args,
)

if __name__ == "__main__":
    # =========================== Parse arguments =========================== #
    # By increasing priority order, the arguments work as follows:
    #
    #   1. Default value of --dummy_arg=42 is the base value.
    #
    #   2. If dummy_arg is present in either the encoder configuration file
    #     (--enc_cfg) or the decoder configuration file (--dec_cfg), then it
    #     overrides the default value.
    #
    #   3. If --dummy_arg is explicitly provided in the command line, then it
    #      overrides both the default value and the value listed in the
    #      configuration file.

    parser = configargparse.ArgumentParser()
    # -------- These arguments are not in the configuration files
    parser.add(
        "-i",
        "--input",
        help="Path of the input image. Either .png or .yuv",
        type=str,
    )
    parser.add(
        "-o",
        "--output",
        help="Path of the compressed bitstream. If empty, no bitstream is written",
        type=str,
        default="",
    )

    parser.add("--workdir", help="Path of the working_directory", type=str, default=".")
    parser.add("--lmbda", help="Rate constraint", type=float, default=1e-3)
    parser.add(
        "--job_duration_min",
        type=int,
        default=-1,
        help="Exit and save the encoding after this duration. Use -1 to only exit at the end.",
    )

    parser.add_argument(
        "--print_detailed_archi",
        action="store_true",
        help="Print detailed NN architecture",
    )
    parser.add_argument(
        "--print_detailed_struct",
        action="store_true",
        help="Print detailed Coding structure",
    )

    # -------- Configuration files
    parser.add("--enc_cfg", is_config_file=True, help="Encoder config file")
    parser.add(
        "--dec_cfg_residue",
        is_config_file=True,
        help="Residue (or intra) decoder config file",
    )
    parser.add(
        "--dec_cfg_motion", is_config_file=True, help="Motion decoded config file"
    )

    # -------- These arguments are in the configuration files

    # ==== Encoder-side arguments

    parser.add("--start_lr", help="Initial learning rate", type=float, default=1e-2)
    parser.add(
        "--n_itr",
        help="Number of iterations for the main training stage. Use this to "
        "change the duration of the encoding.",
        type=int,
        default=int(1e4),
    )
    parser.add(
        "--n_itr_pretrain_motion",
        help="Number of iterations for the motion pre-training stage.",
        type=int,
        default=int(1e3),
    )

    parser.add("--n_train_loops", help="Number of training loops", type=int, default=1)
    parser.add(
        "--preset",
        help='Recipe type. Either "c3x" or "debug".',
        type=str,
        default="c3x",
    )

    # ==== Decoder-side arguments
    parser.add(
        "--layers_synthesis_residue",
        type=str,
        default="48-1-linear-relu,X-1-linear-none,X-3-residual-relu,X-3-residual-none",
        help="Syntax example: "
        " "
        "    12-1-linear-relu,12-1-residual-relu,X-1-linear-relu,X-3-residual-none "
        " "
        "    This is a 4 layers synthesis. Each layer is separated by comas and is "
        "described using the following syntax: "
        " "
        "    <output_dim>-<kernel_size>-<type>-<non_linearity>. "
        " "
        "<output_dim> is the number of output features. If set to X, this is replaced by the "
        "number of required output features i.e. 3 for a RGB or YUV frame or 4 for a P-frame "
        "residue + alpha. "
        " "
        "<kernel_size> is the spatial dimension of the kernel. Use 1 to mimic an MLP. "
        " "
        "<type> is either 'linear' for a standard conv or 'residual' for a convolution "
        "with a residual connection block i.e. layer(x) = x + conv(x). "
        " "
        "<non_linearity> Can be 'none' for no non-linearity, 'relu' for a ReLU "
        "non-linearity. "
        " Parameterize the residue decoder.",
    )
    parser.add(
        "--layers_synthesis_motion",
        type=str,
        default="9-1-linear-relu,X-1-linear-none,X-3-residual-none",
        help="Identical to --layers_synthesis_residue but for the motion decoder.",
    )

    parser.add(
        "--arm_residue",
        type=str,
        default="16,2",
        help="<arm_context_and_layer_dimension>,<number_of_hidden_layers> "
        "First number indicates both the context size **AND** the hidden layer dimension. "
        "Second number indicates the number of hidden layer(s). 0 gives a linear ARM module. "
        " Parameterize the residue decoder.",
    )
    parser.add(
        "--arm_motion",
        type=str,
        default="8,1",
        help="Identical to --arm_residue but for the motion decoder.",
    )

    parser.add(
        "--n_ft_per_res_residue",
        type=str,
        default="1,1,1,1,1,1,1",
        help="Number of feature for each latent resolution. e.g. "
        " --n_ft_per_res_residue=1,1,1,1,1,1,1 "
        " for 7 latent grids with variable resolutions. "
        " Parameterize the residue decoder.",
    )
    parser.add(
        "--n_ft_per_res_motion",
        type=str,
        default="0,0,1,1,1,1,1",
        help="Identical to --n_ft_per_res_residue but for the motion decoder.",
    )

    parser.add(
        "--ups_k_size_residue",
        type=int,
        default=8,
        help="Upsampling kernel size for the transposed convolutions. "
        "Must be even and >= 4. Parameterize the residue decoder.",
    )
    parser.add(
        "--ups_k_size_motion",
        type=int,
        default=8,
        help="Identical to --ups_k_size_residue but for the motion decoder.",
    )

    parser.add(
        "--ups_preconcat_k_size_residue",
        type=int,
        default=7,
        help="Upsampling kernel size for the pre-concatenation convolutions. "
        "Must be odd. Parameterize the residue decoder.",
    )
    parser.add(
        "--ups_preconcat_k_size_motion",
        type=int,
        default=7,
        help="Identical to --ups_preconcat_k_size_residue but for the motion decoder.",
    )

    parser.add(
        "--warp_filter_size",
        type=int,
        default=8,
        help="Number of taps for the warping interpolation filter. "
    )

    args = parser.parse_args()
    print(args)
    print("----------")
    # useful for logging where different settings came from
    print(parser.format_values())
    # =========================== Parse arguments =========================== #

    # =========================== Parse arguments =========================== #
    os.chdir(args.workdir)


    start_print = (
        "\n\n"
        "*----------------------------------------------------------------------------------------------------------*\n"
        "|                                                                                                          |\n"
        "|                                                                                                          |\n"
        "|       ,gggg,                                                                                             |\n"
        '|     ,88"""Y8b,                           ,dPYb,                             ,dPYb,                       |\n'
        "|    d8\"     `Y8                           IP'`Yb                             IP'`Yb                       |\n"
        "|   d8'   8b  d8                           I8  8I                             I8  8I      gg               |\n"
        "|  ,8I    \"Y88P'                           I8  8'                             I8  8'      \"\"               |\n"
        "|  I8'             ,ggggg,      ,ggggg,    I8 dP      aaaaaaaa        ,gggg,  I8 dPgg,    gg     ,gggg,    |\n"
        '|  d8             dP"  "Y8ggg  dP"  "Y8ggg I8dP       """"""""       dP"  "Yb I8dP" "8I   88    dP"  "Yb   |\n'
        "|  Y8,           i8'    ,8I   i8'    ,8I   I8P                      i8'       I8P    I8   88   i8'         |\n"
        "|  `Yba,,_____, ,d8,   ,d8'  ,d8,   ,d8'  ,d8b,_                   ,d8,_    _,d8     I8,_,88,_,d8,_    _   |\n"
        '|    `"Y8888888 P"Y8888P"    P"Y8888P"    8P\'"Y88                  P""Y8888PP88P     `Y88P""Y8P""Y8888PP   |\n'
        "|                                                                                                          |\n"
        "|                                                                                                          |\n"
        "| version 4.1.0, July 2025                                                              © 2023-2025 Orange |\n"
        "*----------------------------------------------------------------------------------------------------------*\n"
    )
    print(start_print)
    image_save_prefix = "0000-"

    # Dump raw parameters into a text file to keep track
    with open(f"{image_save_prefix}param.txt", "w") as f_out:
        f_out.write(str(args))
        f_out.write("\n")
        f_out.write("----------\n")
        f_out.write(
            parser.format_values()
        )  # useful for logging where different settings came from

    # Successively parse the Cool-chic architectures for the residue
    # and the motion Cool-chic
    coolchic_enc_param = {
        cc_name: CoolChicEncoderParameter(
            **get_coolchic_param_from_args(args, cc_name)
        )
        for cc_name in ["residue", "motion"]
    }
    frame_encoder_manager = FrameEncoderManager(**get_manager_from_args(args))
    warp_parameter = WarpParameter(**get_warp_param_from_args(args))

    # Automatic device detection
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    print(f"{'Device':<26}: {device}")

    coding_structure = CodingStructure(**get_coding_structure_from_args(args))
    print(
        f"\n{coding_structure.pretty_string(print_detailed_struct=args.print_detailed_struct)}\n"
    )

    exit_code = encode_one_frame(
        video_path=IMAGE_PATHS[0],
        coding_structure=coding_structure,
        coolchic_enc_param=coolchic_enc_param,
        frame_encoder_manager=frame_encoder_manager,
        warp_parameter=warp_parameter,
        coding_index=args.coding_idx,
        job_duration_min=args.job_duration_min,
        device=device,
        print_detailed_archi=args.print_detailed_archi,
    )

    # Bitstream
    if args.output != "":
        from enc.bitstream.encode import encode_frame

        frame_enc_path = (
            f"{image_save_prefix}frame_encoder.pt"
        )
        frame_encoder, _ = load_frame_encoder(frame_enc_path)
        encode_frame(
            frame_encoder, None, args.output, frame.display_order, hls_sig_blksize=16
        )


# Cool-Chic Encoding Pipeline Explained

## Overview of the Encoding Process

The Cool-Chic encoding pipeline consists of several key phases:

### 1. **Initialization Phase**
- Load image and convert to Cool-Chic format
- Initialize `FrameEncoder` with:
  - `CoolChicEncoder` modules (for "residue" and optionally "motion")  
  - Neural network architectures (ARM, Upsampling, Synthesis)
  - Training parameters and quantization settings

### 2. **Training Phase** (Rate-Distortion Optimization)
- **Input**: Original image + λ (rate constraint)
- **Process**: Iterative optimization to minimize `Loss = MSE + λ × Rate`
- **Components trained**:
  - **Latent Variables**: Hierarchical grids at different resolutions
  - **ARM (Auto-Regressive Module)**: Predicts probability distributions for latents
  - **Upsampling Network**: Takes latents from coarse to fine resolution
  - **Synthesis Network**: Converts upsampled features to final image pixels

### 3. **Quantization Phase**
- Convert trained neural networks to integer representations
- Quantize latent variables for entropy coding
- Choose optimal quantization steps for rate-distortion trade-off

### 4. **Bitstream Generation Phase**
- **GOP Header**: Image dimensions, bitdepth, frame type
- **Frame Header**: Quantization indices, network topology flags
- **Neural Network Weights**: Entropy-coded quantized weights/biases
- **Latent Variables**: Entropy-coded using ARM-predicted distributions

## Key Components for Image Compression

For **image compression** (I-frames), you need:

1. **Latent Variables** (`CoolChicEncoder.latent_grids`)
2. **ARM Network** (`CoolChicEncoder.arm`) - predicts latent distributions  
3. **Upsampling Network** (`CoolChicEncoder.upsampling`) - upscales latents
4. **Synthesis Network** (`CoolChicEncoder.synthesis`) - **THIS IS KEY FOR LOSSLESS**
5. **Entropy Coder** (CABAC implementation in C++)

## What Changes for Lossless Compression

**Current (Lossy)**: Synthesis → RGB pixel values directly  
**Lossless Target**: Synthesis → Logistic distribution parameters (μ, σ) → Entropy decode pixels

Let's now implement this manually step by step...

In [ ]:
# Manual Cool-Chic Encoding Implementation
# Let's break down the encoding process step by step

print("=" * 80)
print("MANUAL COOL-CHIC ENCODING PIPELINE")
print("=" * 80)

# Step 1: Initialize the components manually
print("\n1. INITIALIZING COOL-CHIC COMPONENTS")
print("-" * 40)

from enc.component.coolchic import CoolChicEncoder, CoolChicEncoderParameter
from enc.component.frame import FrameEncoder
from enc.component.intercoding.warp import WarpParameter
from enc.utils.misc import COOLCHIC_ENCODER_CONFIGURATION

# Device setup
device = "cpu"  # Use CPU for clarity
print(f"Device: {device}")

# Image setup - let's work with the first Kodak image
target_image = frame_data.data.unsqueeze(0)  # Add batch dimension
img_size = (frame_data.data.shape[-2], frame_data.data.shape[-1])
print(f"Target image shape: {target_image.shape}")
print(f"Image size: {img_size}")

# Configure the Cool-Chic encoder parameters
coolchic_param = CoolChicEncoderParameter(
    img_size=img_size,
    # Latent configuration
    n_ft_per_res=[1, 1, 1, 1, 1, 1],  # 6 resolution levels with 1 feature each
    # ARM configuration  
    dim_arm=16,                       # ARM dimension
    n_hidden_layers_arm=2,            # ARM hidden layers
    # Upsampling configuration
    ups_k_size=8,                     # Upsampling kernel size
    ups_preconcat_k_size=7,           # Pre-concatenation kernel size
    # Synthesis configuration - THIS IS KEY FOR LOSSLESS
    layers_synthesis="48-1-linear-relu,X-1-linear-none,X-3-residual-relu,X-3-residual-none",
    encoder_gain=16,                  # Encoder gain
)

print(f"Cool-Chic parameters configured")
print(f"  - Latent grids: {len(coolchic_param.n_ft_per_res)} levels")
print(f"  - ARM dimension: {coolchic_param.dim_arm}")  
print(f"  - Synthesis layers: {coolchic_param.layers_synthesis}")

# Create the CoolChic encoder for residue (intra coding)
coolchic_encoder = CoolChicEncoder(coolchic_param).to(device)
print(f"CoolChic encoder created with {sum(p.numel() for p in coolchic_encoder.parameters())} parameters")

# Print the architecture
print(f"\nCOOL-CHIC ARCHITECTURE:")
print(f"  ARM: {coolchic_encoder.arm}")
print(f"  Upsampling: {coolchic_encoder.upsampling}")
print(f"  Synthesis: {coolchic_encoder.synthesis}")

# Show latent grid shapes
print(f"\nLATENT GRIDS:")
for i, grid in enumerate(coolchic_encoder.latent_grids):
    print(f"  Grid {i}: {grid.data.shape}")

print("\n✓ Step 1 Complete: Components initialized")

In [ ]:
# Step 2: Manual Forward Pass - Understanding Each Component
print("\n2. MANUAL FORWARD PASS BREAKDOWN")
print("-" * 40)

# Put encoder in training mode
coolchic_encoder.train()

print("Forward pass through individual components:")

# 2a. Latent variables (the learned representation)
print("\n2a. LATENT VARIABLES:")
latent_grids = [grid.data for grid in coolchic_encoder.latent_grids]
for i, grid in enumerate(latent_grids):
    print(f"  Grid {i}: shape={grid.shape}, mean={grid.mean():.4f}, std={grid.std():.4f}")

# 2b. ARM processing (predicts latent distributions for entropy coding)
print("\n2b. ARM (Auto-Regressive Module):")
print("  → Predicts probability distributions for each latent pixel")
print("  → Uses causal context (previously coded latents)")
print("  → Outputs: μ (mean) and log_scale for Laplace distribution")

# Simulate ARM processing manually
with torch.no_grad():
    # ARM takes spatial context and outputs distribution parameters
    # This is a simplified view - real ARM processes latents sequentially
    total_arm_rate = 0
    for i, grid in enumerate(latent_grids):
        if grid.numel() > 0:  # Only process non-empty grids
            # ARM would process this grid to get μ and log_scale
            dummy_context = torch.zeros(1, coolchic_encoder.param.dim_arm, device=device)
            arm_output = coolchic_encoder.arm(dummy_context)  # [1, 2] -> [μ, log_scale]
            
            # Rate calculation (simplified)
            grid_rate = grid.numel() * 1.5  # Estimated bits per latent
            total_arm_rate += grid_rate
            print(f"  Grid {i} ARM output shape: {arm_output.shape}, estimated rate: {grid_rate:.1f} bits")
    
print(f"  Total estimated latent rate: {total_arm_rate:.1f} bits")

# 2c. Upsampling (from coarse latents to full resolution)
print("\n2c. UPSAMPLING NETWORK:")
print("  → Takes hierarchical latents and upsamples to full resolution")
print("  → Uses transposed convolutions with learned kernels")

upsampled_features = coolchic_encoder.upsampling(latent_grids)
print(f"  Input: {len(latent_grids)} grids at different resolutions")
print(f"  Output: {upsampled_features.shape} (full resolution features)")

# 2d. Synthesis (converts features to pixels - KEY FOR LOSSLESS)
print("\n2d. SYNTHESIS NETWORK (KEY FOR LOSSLESS):")
print("  → Current: Outputs RGB pixel values directly")
print("  → For lossless: Should output logistic distribution parameters")

synthesized_output = coolchic_encoder.synthesis(upsampled_features)
print(f"  Current output shape: {synthesized_output.shape}")
print(f"  Current output range: [{synthesized_output.min():.3f}, {synthesized_output.max():.3f}]")
print(f"  Expected for lossless: {synthesized_output.shape[:-1] + (6,)} (μ,σ for each RGB channel)")

print("\n✓ Step 2 Complete: Forward pass breakdown")

In [ ]:
# Step 3: Training Loop (Rate-Distortion Optimization)
print("\n3. TRAINING LOOP (RATE-DISTORTION OPTIMIZATION)")
print("-" * 40)

from enc.training.loss import loss_function

# Setup optimizer
optimizer = torch.optim.Adam(coolchic_encoder.parameters(), lr=1e-3)
lmbda = 1e-3  # Rate constraint

print(f"Optimizer: Adam with lr=1e-3")
print(f"Lambda (rate constraint): {lmbda}")
print(f"Target image shape: {target_image.shape}")

# Training iteration
print("\nPerforming manual training iterations...")

# Store loss history  
loss_history = []
psnr_history = []

for iteration in range(5):  # Just a few iterations for demonstration
    optimizer.zero_grad()
    
    # Forward pass through CoolChic encoder
    coolchic_output = coolchic_encoder()
    decoded_image = coolchic_output.decoded_image
    rate = coolchic_output.rate
    
    # Compute loss (MSE + lambda * Rate)
    loss_output = loss_function(
        decoded_image=decoded_image,
        rate_latent_bit=rate,
        target_image=target_image,
        lmbda=lmbda,
        compute_logs=True
    )
    
    # Backward pass
    loss_output.loss.backward()
    
    # Gradient clipping
    torch.nn.utils.clip_grad_norm_(coolchic_encoder.parameters(), 1e-1)
    
    # Optimizer step
    optimizer.step()
    
    # Compute PSNR
    mse = loss_output.mse
    psnr = -10 * torch.log10(torch.tensor(mse + 1e-10))
    
    # Store history
    loss_history.append(loss_output.loss.item())
    psnr_history.append(psnr.item())
    
    print(f"  Iteration {iteration+1}:")
    print(f"    MSE Loss: {loss_output.mse:.6f}")
    print(f"    Rate (bpp): {loss_output.rate_latent_bpp:.6f}")
    print(f"    Total Loss: {loss_output.loss:.6f}")
    print(f"    PSNR: {psnr:.2f} dB")

print(f"\nTraining summary:")
print(f"  Initial PSNR: {psnr_history[0]:.2f} dB")
print(f"  Final PSNR: {psnr_history[-1]:.2f} dB")
print(f"  PSNR improvement: {psnr_history[-1] - psnr_history[0]:.2f} dB")

print("\n✓ Step 3 Complete: Training loop executed")

In [ ]:
# Step 4: Manual Decoding Process & Analysis for Lossless Modification
print("\n4. MANUAL DECODING PROCESS & LOSSLESS ANALYSIS")
print("-" * 40)

# Set encoder to evaluation mode
coolchic_encoder.eval()

print("Current Cool-Chic Decoding Process:")
print("1. Load latent variables and network weights from bitstream")
print("2. ARM predicts latent probability distributions")  
print("3. Entropy decode latents using ARM predictions")
print("4. Upsampling takes latents to full resolution")
print("5. Synthesis outputs RGB pixel values directly")

# Simulate the decoding process
with torch.no_grad():
    print("\nSimulating decoding process:")
    
    # Step 1: Latent variables (would be entropy decoded in real decoder)
    print("  1. Latent variables loaded:")
    for i, grid in enumerate(coolchic_encoder.latent_grids):
        print(f"     Grid {i}: {grid.data.shape}")
    
    # Step 2: ARM processing (would use actual entropy decoder)
    print("  2. ARM predicts latent distributions (μ, log_scale)")
    
    # Step 3: Upsampling
    latent_grids = [grid.data for grid in coolchic_encoder.latent_grids]
    upsampled = coolchic_encoder.upsampling(latent_grids)
    print(f"  3. Upsampled features: {upsampled.shape}")
    
    # Step 4: Synthesis (THIS IS WHERE LOSSLESS CHANGES)
    final_output = coolchic_encoder.synthesis(upsampled)
    print(f"  4. Synthesis output: {final_output.shape}")
    print(f"     Range: [{final_output.min():.3f}, {final_output.max():.3f}]")
    
    # Clamp to valid range
    decoded_pixels = torch.clamp(final_output, 0.0, 1.0)
    print(f"  5. Final decoded pixels: {decoded_pixels.shape}")

print("\n" + "="*60)
print("ANALYSIS FOR LOSSLESS MODIFICATION")
print("="*60)

print("\n📋 WHAT TO KEEP (Core Components):")
print("  ✓ Latent Variables - Still need hierarchical representation")
print("  ✓ ARM - Still predicts latent distributions for entropy coding")  
print("  ✓ Upsampling - Still need to go from coarse to fine resolution")
print("  ✓ Entropy Coding - CABAC implementation for compression")

print("\n🔄 WHAT TO MODIFY (Synthesis Network):")
current_synthesis = coolchic_encoder.synthesis
print(f"  Current synthesis: {current_synthesis}")
print(f"  Current output: {final_output.shape} (RGB values)")
print(f"  Modified output needed: {final_output.shape[:-1] + (6,)} (μ,σ for R,G,B)")
print("  → Change final layer from 3 → 6 output channels")
print("  → First 3 channels: μ (mean) for R,G,B") 
print("  → Last 3 channels: log_σ (log scale) for R,G,B")

print("\n🔄 WHAT TO REPLACE (Loss Function):")
print("  Current: MSE + λ × Rate_latents")
print("  New: -LogLikelihood(pixels|logistic) + λ × Rate_latents")
print("  → Negative log-likelihood of target pixels under predicted logistic distributions")

print("\n➕ WHAT TO ADD (Pixel Entropy Coding):")
print("  Current: Direct pixel output")
print("  New: Sample pixels from logistic distributions using entropy decoder")
print("  → This ensures perfect reconstruction (lossless)")

print("\n🎯 KEY INSIGHT:")
print("  The entire latent processing pipeline stays the same!")
print("  Only change: How final pixels are generated")
print("  From: Direct prediction → To: Probabilistic modeling + entropy coding")

print("\n✓ Step 4 Complete: Decoding analysis and lossless roadmap")

In [ ]:
# Step 5: Prototype Lossless Components
print("\n5. PROTOTYPE LOSSLESS COMPONENTS")
print("-" * 40)

print("Let's implement the key components needed for lossless compression:")

# 5a. Logistic Distribution Functions
print("\n5a. LOGISTIC DISTRIBUTION FUNCTIONS:")

def logistic_log_prob(x, mu, sigma):
    """Compute log probability of x under logistic distribution"""
    eps = 1e-7  # For numerical stability
    sigma = torch.clamp(sigma, min=eps)
    z = (x - mu) / sigma
    return -z - torch.log(sigma) - 2 * torch.log1p(torch.exp(-torch.abs(z)))

def logistic_sample(mu, sigma, shape):
    """Sample from logistic distribution (for entropy decoding simulation)"""
    eps = 1e-7
    sigma = torch.clamp(sigma, min=eps)
    # Use uniform random and inverse CDF
    u = torch.rand(shape)
    # Inverse CDF of logistic: μ + σ * log(u / (1-u))
    return mu + sigma * torch.log(u / (1 - u + eps))

print("  ✓ logistic_log_prob() - for loss computation")
print("  ✓ logistic_sample() - for entropy decoding simulation")

# 5b. Modified Synthesis Network (conceptual)
print("\n5b. LOSSLESS SYNTHESIS CONCEPT:")

# Current synthesis output
with torch.no_grad():
    current_output = coolchic_encoder.synthesis(upsampled)
    print(f"  Current synthesis output: {current_output.shape}")
    
    # Simulate lossless synthesis (double the channels)
    # In practice, you'd modify the final layer of synthesis network
    B, C, H, W = current_output.shape
    lossless_output = torch.randn(B, C*2, H, W)  # Simulate 6 channels for RGB
    
    # Split into mu and log_sigma
    mu = lossless_output[:, :C, :, :]      # First 3 channels: mean
    log_sigma = lossless_output[:, C:, :, :] # Last 3 channels: log scale
    sigma = torch.exp(log_sigma)
    
    print(f"  Lossless synthesis output: {lossless_output.shape}")
    print(f"  μ (mean): {mu.shape}, range=[{mu.min():.3f}, {mu.max():.3f}]")
    print(f"  σ (scale): {sigma.shape}, range=[{sigma.min():.3f}, {sigma.max():.3f}]")

# 5c. Lossless Loss Function
print("\n5c. LOSSLESS LOSS FUNCTION:")

def lossless_loss_function(mu, sigma, target_image, rate_latent_bit, lmbda=1e-3):
    """Lossless loss: negative log-likelihood + latent rate"""
    
    # Negative log-likelihood of target pixels under logistic distribution
    nll = -logistic_log_prob(target_image, mu, sigma).sum()
    
    # Rate of latents (same as before)
    latent_rate = sum(rate_latent_bit.values()) if isinstance(rate_latent_bit, dict) else rate_latent_bit
    
    # Total loss
    loss = nll + lmbda * latent_rate
    
    return {
        'loss': loss,
        'nll': nll,
        'rate': latent_rate,
        'nll_bpp': nll / (target_image.shape[-2] * target_image.shape[-1]),
        'rate_bpp': latent_rate / (target_image.shape[-2] * target_image.shape[-1])
    }

# Test the lossless loss
loss_result = lossless_loss_function(mu, sigma, target_image, 1000.0, lmbda=1e-3)
print(f"  Lossless loss: {loss_result['loss']:.2f}")
print(f"  NLL (bpp): {loss_result['nll_bpp']:.4f}")
print(f"  Rate (bpp): {loss_result['rate_bpp']:.4f}")

# 5d. Entropy Decoding Simulation
print("\n5d. ENTROPY DECODING SIMULATION:")

# Simulate sampling pixels from the predicted distributions
sampled_pixels = logistic_sample(mu, sigma, mu.shape)
sampled_pixels = torch.clamp(sampled_pixels, 0.0, 1.0)  # Clamp to valid range

print(f"  Sampled pixels: {sampled_pixels.shape}")
print(f"  Range: [{sampled_pixels.min():.3f}, {sampled_pixels.max():.3f}]")

# Check reconstruction error (should be 0 for true lossless)
reconstruction_error = torch.abs(sampled_pixels - target_image).mean()
print(f"  Reconstruction error: {reconstruction_error:.6f}")
print("  Note: In true lossless, this would be exactly 0")

print("\n" + "="*60)
print("SUMMARY: LOSSLESS TRANSFORMATION ROADMAP")
print("="*60)

print("\n1. MODIFY SYNTHESIS NETWORK:")
print("   - Change final layer: 3 → 6 output channels")
print("   - Output logistic parameters instead of pixel values")

print("\n2. REPLACE LOSS FUNCTION:")
print("   - From: MSE + λ × Rate")  
print("   - To: -LogLikelihood + λ × Rate")

print("\n3. ADD ENTROPY CODING FOR PIXELS:")
print("   - Use predicted logistic distributions")
print("   - Sample pixels during decoding for perfect reconstruction")

print("\n4. KEEP EVERYTHING ELSE:")
print("   - Latent variables and ARM for context modeling")
print("   - Upsampling network for multi-resolution processing")
print("   - CABAC entropy coder for latents")

print("\n✓ Step 5 Complete: Lossless prototype implemented")
print("✓ Manual encoding pipeline analysis complete!")
print("\nNext step: Implement the actual synthesis network modification...")